In [1]:
%pip install transformers==4.31.0
%pip install peft==0.4.0
%pip install accelerate==0.21.0
#%pip install bitsandbytes==0.40.2
%pip install safetensors==0.3.3 # remove this?
%pip install tokenizers==0.13.3 # remove this?
%pip install datasets==2.14.1


^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached datasets-2.14.1-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.14.1-py3-none-any.whl (492 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import argparse
import os
import argparse
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    default_data_collator,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
import torch
parser = argparse.ArgumentParser()

# add model id and dataset path argument
parser.add_argument(
    "--model_id",
    type=str,
    default="NousResearch/Llama-2-7b-hf", # not gated
    help="Model id to use for training.",
)
parser.add_argument(
    "--dataset_path", 
    type=str, 
    default="lm_dataset", 
    help="Path to dataset."
)
# add training hyperparameters for epochs, batch size, learning rate, and seed
parser.add_argument(
    "--epochs", 
    type=int, 
    default=1, 
    help="Number of epochs to train for."
)
parser.add_argument(
    "--per_device_train_batch_size",
    type=int,
    default=1,
    help="Batch size to use for training.",
)
parser.add_argument(
    "--lr", 
    type=float, 
    default=5e-5, 
    help="Learning rate to use for training."
)
parser.add_argument(
    "--seed", 
    type=int, 
    default=42, 
    help="Seed to use for training."
)
parser.add_argument(
    "--gradient_checkpointing",
    type=bool,
    default=True,
    help="Path to deepspeed config file.",
)
parser.add_argument(
    "--bf16",
    type=bool,
    default=True if torch.cuda.get_device_capability()[0] >= 8 else False,
    help="Whether to use bf16.",
)
parser.add_argument(
    "--merge_weights",
    type=bool,
    default=True,
    help="Whether to merge LoRA weights with base model.",
)
args, _ = parser.parse_known_args()
# Define training args
output_dir = "./tmp/llama2_lora"

2024-03-13 14:39:27.228989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 14:39:27.229057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 14:39:27.230483: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:215: UserWarning: 
NVIDIA H100 PCIe with CUDA capability sm_90 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_60 sm_70 sm_75 compute_70 compute_75.
If you want to use the NVIDIA H100 PCIe GPU with PyTorch, please check the instruction

In [2]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [15]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj):

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Router(nn.Module) :
    def __init__(self, input_dim, num_experts):
        super().__init__()
        self.ff = nn.Linear (input_dim, num_experts)
    def forward (self, x):
        logits = self.ff(x)
        probs = F.softmax(logits, dim=-1)
        return logits, probs

In [5]:
class MoV(nn.Module):
    def __init__(self, linear_layer, num_experts):
        super (MoV, self).__init__()
        # Original linear layer
        self.original = linear_layer
        self.router = Router(self.original.in_features, num_experts)
        self.experts = nn.Parameter(torch.ones(num_experts, linear_layer.out_features))
    def prepare_model_gradients(self):
        self.experts.requires_grad_(True)
        self.router.ff.weight.requires_grad_(True)
    def forward (self, x):
        frozen_output = self.original(x)
        _, gating_probs = self.router(x)
        # Compute the weighted sum of expert outputs
        mov_combined = torch.einsum("bse,ed->bsd", gating_probs, self.experts)
        return frozen_output * mov_combined

In [16]:
def create_peft_model(model, gradient_checkpointing=True, bf16=True):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
    )
    from peft.tuners.lora import LoraLayer

    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # # get lora target modules
    # modules = find_all_linear_names(model)    
    
    #If only targeting attention blocks of the model
    modules = ["q_proj", "v_proj"]

    #If targeting all linear layers
    # modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj'] #,'lm_head']

    print(f"Found {len(modules)} modules to quantize: {modules}")

    peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        target_modules=modules,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )

    model = get_peft_model(model, peft_config)

    model.print_trainable_parameters()
    return model


In [17]:
model = AutoModelForCausalLM.from_pretrained(
    args.model_id,
    use_cache=False
    if args.gradient_checkpointing
    else True,  # this is needed for gradient checkpointing
    device_map="auto",
)

# create peft config
model = create_peft_model(
    model, gradient_checkpointing=args.gradient_checkpointing, bf16=args.bf16
)
# model.config.use_cache = False
# model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Found 2 modules to quantize: ['q_proj', 'v_proj']
trainable params: 33,554,432 || all params: 6,771,970,048 || trainable%: 0.49548996469513035


In [19]:
import re
# import attrgetter
from operator import attrgetter
def adapt_model_with_moe_peft(model, experts):
    # Only modify the key/value and linear activations
    llama_regex_match = "(.*(self_attn|LlamaAttention).(k_proj|v_proj).weight)|(.*LlamaMLP.down_proj.weight)"
    
    for n, _ in model.named_parameters():
        if re.search(llama_regex_match, n) is None:
            continue
        # Get module that the parameter belongs to
        module_name = ".".join(n.split(".")[:-1])
        module = attrgetter(module_name)(model)
        module_parent_name = ".".join(n.split(".")[:-2])
        module_key_name = n.split(".")[-2]
        module_parent = attrgetter(module_parent_name)(model)
        setattr(module_parent, module_key_name, MoV(module, experts))

    # Freeze base model and set MoV weights as tunable
    for m in model.modules():
        m.requires_grad_(False)
        if isinstance(m, MoV):
            m.prepare_model_gradients()
            
num_experts = 100
adapt_model_with_moe_peft(model, num_experts)
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): MoV(
                (original): Linear(in_features=4096, out_features=4096, bi

In [20]:
# count total number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {num_params}")
# count number of trainable parameters
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {num_trainable_params}")

Total number of parameters: 6824405248
Total number of trainable parameters: 26214400


In [21]:
tokenizer = AutoTokenizer.from_pretrained(args.model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [22]:
# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty        
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    # if use_4bit:
    #     trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

In [23]:
# set seed
set_seed(args.seed)

from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
dataset = dataset.select(range(1000))

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# dataset size: 15011

dataset size: 1000
{'instruction': 'Give me a list of some of the most popular song from 70s Japanese Pop singer, Mariya Takeuchi', 'context': '', 'response': '1. Plastic Love\n2. Stay with Me\n3. September\n4. Miracle Love\n5. Yume No Tsuzuki', 'category': 'brainstorming'}


In [24]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

In [25]:

from random import randint
from itertools import chain
from functools import partial

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# print random sample
print(dataset[randint(0, len(dataset))]["text"])

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

### Instruction
What is the Research Collaboratory for Structural Bioinformatics Protein Data Bank (RCSB PDB)?

### Answer
The Research Collaboratory for Structural Bioinformatics Protein Data Bank (RCSB PDB) is a database that provides a wealth of information about the 3D structures of proteins, nucleic acids, and other macromolecules. The database contains experimentally determined atomic coordinates for a large number of macromolecules, which can be used to study their structures, functions, and interactions. The RCSB PDB is widely used in genomics research and drug discovery, as it provides a valuable resource for understanding the structural basis of many biological processes and for designing new drugs that target specific macromolecules.

In addition to the atomic coordinates, the RCSB PDB contains a wealth of additional information about each macromolecule, including experimental methods used for structure determination, citations to relevant scientific literature, and informat

In [26]:
# Define training args
output_dir = "./tmp/llama2_lora"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=args.per_device_train_batch_size,
    bf16=args.bf16,  # Use BF16 if available
    learning_rate=args.lr,
    num_train_epochs=args.epochs,
    gradient_checkpointing=args.gradient_checkpointing,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    data_collator=default_data_collator,
)

# Start training
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,1.747700
20,1.696200
30,1.648800
40,1.595900
50,1.766000
60,1.674100
70,1.589700
80,1.744700
90,1.827400
100,1.638700


TrainOutput(global_step=105, training_loss=1.6943521772112164, metrics={'train_runtime': 90.4527, 'train_samples_per_second': 1.161, 'train_steps_per_second': 1.161, 'total_flos': 8636006289899520.0, 'train_loss': 1.6943521772112164, 'epoch': 1.0})